In [6]:
# libraries
import os
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression # logistic regression
from sklearn.model_selection import train_test_split # for train-test split
from sklearn.preprocessing import OneHotEncoder, StandardScaler # for categorical encoding
from sklearn.compose import ColumnTransformer # for combining transformations
from sklearn.pipeline import Pipeline # for creating a pipeline
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report # for evaluation metrics
from sklearn.metrics import roc_auc_score, roc_curve, brier_score_loss # for ROC curve and AUC
import matplotlib.pyplot as plt # for plotting
from sklearn.model_selection import cross_val_predict # for cross-validation predictions
from sklearn.linear_model import LogisticRegressionCV # for cross-validated logistic regression
from sklearn.model_selection import StratifiedKFold # for stratified fold cross-validation
from sklearn.base import clone # for cloning models
from sklearn.model_selection import RepeatedStratifiedKFold # for repeated stratified fold cross-validation
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

import joblib








In [8]:

# Load models
ari_model = joblib.load("BAARD_arp_clin_model.joblib")
bup_model = joblib.load("BAARD_bup_clin_model.joblib")

# Expected features (from training). If your models have feature_names_in_, use those.
ari_features = list(getattr(ari_model, "feature_names_in_", [])) or [
    'age', 'sex', 'edu_lvl', 'baseline_madrs',  'BMI_extreme'  ,'mini_addtl_q1','athf_f1_total_trials_v2','years_with_depression'
]
bup_features = list(getattr(bup_model, "feature_names_in_", [])) or [
   'age', 'sex', 'edu_lvl', 'baseline_madrs',  'BMI_extreme'  ,'mini_addtl_q1','athf_f1_total_trials_v2','years_with_depression'
]


In [9]:
# Load data
master_df = pd.read_csv('C:\\Users\\Hassan\\Documents\\Projects\\baard\\baard_master_sheet.csv')


# make variable had_fall if total_number_falls > 0
master_df['had_fall'] = (master_df['total_number_falls'] > 0).astype(int)

# add new varaible BMI_extremer, where 1 = bmi > 40 or < 20, binary variables
master_df['BMI_extreme'] = ((master_df['bmi'] > 40) | (master_df['bmi'] < 20)).astype(int)

master_df['sex'] = (master_df['gender'] == 'Male').astype(int)


# filter out rows with missing values in the features we care about
ON_arp = master_df[master_df['medication_group'] == "ARIPIPRAZOLE"]
ON_arp = ON_arp.dropna(subset=['remission_status' ,'age', 'sex', 'edu_lvl', 'baseline_madrs',  'BMI_extreme'  ,'mini_addtl_q1','athf_f1_total_trials_v2','years_with_depression'])

ON_bup = master_df[master_df['medication_group'] == "BUPROPION"]
ON_bup = ON_bup.dropna(subset=['remission_status' ,'age', 'sex', 'edu_lvl', 'baseline_madrs',  'BMI_extreme'  ,'mini_addtl_q1','athf_f1_total_trials_v2','years_with_depression'])



In [6]:
def build_matrix(df, features, id_col="record_id"):
    X = df[features].apply(pd.to_numeric, errors="coerce")
    mask = X.notna().all(axis=1)   # use your training-time imputer instead if you had one
    return df.loc[mask, id_col].to_numpy(), X.loc[mask].to_numpy(), mask


In [7]:
# --- ARI model → BUP cohort ---
bup_ids, X_bup_for_ari, m_bup = build_matrix(ON_bup, ari_features)
bup_probs_from_ari = ari_model.predict_proba(X_bup_for_ari)[:, 1]
preds_bup_from_ari = pd.DataFrame({
    "record_id": bup_ids,
    "prob_remission": bup_probs_from_ari
})
# Optional evaluation if labels exist
if "remission_status" in ON_bup.columns:
    y_bup = ON_bup.loc[m_bup, "remission_status"].astype(int).to_numpy()
    print("BUP (scored by ARI model) AUC:", roc_auc_score(y_bup, bup_probs_from_ari))
    print("BUP (scored by ARI model) Brier:", brier_score_loss(y_bup, bup_probs_from_ari))

# --- BUP model → ARI cohort ---
ari_ids, X_ari_for_bup, m_ari = build_matrix(ON_arp, bup_features)
ari_probs_from_bup = bup_model.predict_proba(X_ari_for_bup)[:, 1]
preds_ari_from_bup = pd.DataFrame({
    "record_id": ari_ids,
    "prob_remission": ari_probs_from_bup
})
if "remission_status" in ON_arp.columns:
    y_ari = ON_arp.loc[m_ari, "remission_status"].astype(int).to_numpy()
    print("ARI (scored by BUP model) AUC:", roc_auc_score(y_ari, ari_probs_from_bup))
    print("ARI (scored by BUP model) Brier:", brier_score_loss(y_ari, ari_probs_from_bup))


BUP (scored by ARI model) AUC: 0.6507057987204224
BUP (scored by ARI model) Brier: 0.19240748915104927
ARI (scored by BUP model) AUC: 0.561952861952862
ARI (scored by BUP model) Brier: 0.2745601443434609


In [8]:
# --- ARI model → BUP cohort ---
bup_ids, X_bup_for_ari, m_bup = build_matrix(ON_bup, ari_features)
bup_probs_from_ari = ari_model.predict_proba(X_bup_for_ari)[:, 1]

bup_out = (
    pd.DataFrame({'record_id': bup_ids, 'prob_remission': bup_probs_from_ari})
      .merge(ON_bup.loc[m_bup, ['record_id','age','sex','remission_status']],
             on='record_id', how='left')
      [['record_id','age','sex','remission_status','prob_remission']]
)
# Optional: clearer column name
bup_out = bup_out.rename(columns={'prob_remission':'prob_remission_from_ARI_model'})

# --- BUP model → ARI cohort ---
ari_ids, X_ari_for_bup, m_ari = build_matrix(ON_arp, bup_features)  # <- your ARI cohort is ON_arp
ari_probs_from_bup = bup_model.predict_proba(X_ari_for_bup)[:, 1]

ari_out = (
    pd.DataFrame({'record_id': ari_ids, 'prob_remission': ari_probs_from_bup})
      .merge(ON_arp.loc[m_ari, ['record_id','age','sex','remission_status']],
             on='record_id', how='left')
      [['record_id','age','sex','remission_status','prob_remission']]
)
ari_out = ari_out.rename(columns={'prob_remission':'prob_remission_from_BUP_model'})

# (Optional) combine and tag
all_out = pd.concat([
    bup_out.assign(group='BUP', scored_by='ARI_model'),
    ari_out.assign(group='ARI', scored_by='BUP_model')
], ignore_index=True)

# (Optional) save
bup_out.to_csv('preds_BUP_scored_by_ARI.csv', index=False)
ari_out.to_csv('preds_ARI_scored_by_BUP.csv', index=False)
# all_out.to_csv('preds_crosscohort_all.csv', index=False)


In [81]:
threshold = 0.27
bup_out['y_pred'] = (bup_out['prob_remission_from_ARI_model'] >= threshold).astype(int)
ari_out['y_pred'] = (ari_out['prob_remission_from_BUP_model'] >= threshold).astype(int)


In [83]:
threshold = 0.1
bup_out['y_pred'] = (bup_out['prob_remission_from_ARI_model'] >= threshold).astype(int)
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(bup_out['remission_status'].astype(int),
                                  bup_out['y_pred']).ravel()
sens = tp/(tp+fn) if (tp+fn) else float('nan')
spec = tn/(tn+fp) if (tn+fp) else float('nan')
print(dict(tp=tp, fp=fp, tn=tn, fn=fn, sensitivity=sens, specificity=spec))


{'tp': np.int64(35), 'fp': np.int64(67), 'tn': np.int64(22), 'fn': np.int64(2), 'sensitivity': np.float64(0.9459459459459459), 'specificity': np.float64(0.24719101123595505)}


In [84]:
from sklearn.metrics import confusion_matrix, roc_auc_score, brier_score_loss
import numpy as np

def threshold_report(df, prob_col, y_col='remission_status', thr=0.10):
    y = df[y_col].astype(int).values
    p = df[prob_col].values
    yhat = (p >= thr).astype(int)

    tn, fp, fn, tp = confusion_matrix(y, yhat).ravel()
    n = len(y); pos = int(y.sum()); neg = n - pos

    acc  = (tp + tn) / n
    sens = tp / (tp + fn) if (tp + fn) else np.nan
    spec = tn / (tn + fp) if (tn + fp) else np.nan
    prec = tp / (tp + fp) if (tp + fp) else np.nan
    npv  = tn / (tn + fn) if (tn + fn) else np.nan
    f1   = (2 * prec * sens / (prec + sens)) if (prec > 0 and sens > 0) else np.nan
    bal  = 0.5 * (sens + spec)
    auc  = roc_auc_score(y, p)
    brier = brier_score_loss(y, p)

    print(f"\nThreshold report @ thr = {thr:.2f}")
    print("-" * 40)
    print(f"N={n} | Positives={pos} ({pos/n:.1%}) | Predicted +={tp+fp} ({(tp+fp)/n:.1%})")
    print("\nConfusion matrix (rows=true, cols=pred):")
    print(f"               Pred 0   Pred 1")
    print(f"True 0 ({neg:3d})   {tn:7d} {fp:8d}")
    print(f"True 1 ({pos:3d})   {fn:7d} {tp:8d}")
    print("\nMetrics:")
    print(f"  Accuracy:           {acc:.3f}")
    print(f"  Sensitivity (TPR):  {sens:.3f}")
    print(f"  Specificity (TNR):  {spec:.3f}")
    print(f"  Precision (PPV):    {prec:.3f}")
    print(f"  NPV:                {npv:.3f}")
    print(f"  Balanced Accuracy:  {bal:.3f}")
    print(f"  F1 score:           {f1:.3f}")
    print(f"  AUC (prob-based):   {auc:.3f}")
    print(f"  Brier score:        {brier:.3f}")


In [85]:
threshold_report(bup_out, 'prob_remission_from_ARI_model', 'remission_status', thr=0.10)
# e.g., also try the operating points you care about:
for t in (0.27, 0.29, 0.34):
    threshold_report(bup_out, 'prob_remission_from_ARI_model', thr=t)



Threshold report @ thr = 0.10
----------------------------------------
N=126 | Positives=37 (29.4%) | Predicted +=102 (81.0%)

Confusion matrix (rows=true, cols=pred):
               Pred 0   Pred 1
True 0 ( 89)        22       67
True 1 ( 37)         2       35

Metrics:
  Accuracy:           0.452
  Sensitivity (TPR):  0.946
  Specificity (TNR):  0.247
  Precision (PPV):    0.343
  NPV:                0.917
  Balanced Accuracy:  0.597
  F1 score:           0.504
  AUC (prob-based):   0.709
  Brier score:        0.197

Threshold report @ thr = 0.27
----------------------------------------
N=126 | Positives=37 (29.4%) | Predicted +=69 (54.8%)

Confusion matrix (rows=true, cols=pred):
               Pred 0   Pred 1
True 0 ( 89)        49       40
True 1 ( 37)         8       29

Metrics:
  Accuracy:           0.619
  Sensitivity (TPR):  0.784
  Specificity (TNR):  0.551
  Precision (PPV):    0.420
  NPV:                0.860
  Balanced Accuracy:  0.667
  F1 score:           0.547
  AUC

In [ ]:
import numpy as np, pandas as pd
from sklearn.metrics import confusion_matrix

def sweep_thresholds(df, prob_col, y_col='remission_status'):
    y = df[y_col].astype(int).values
    p = df[prob_col].values
    rows = []
    for t in np.linspace(0, 1, 101):
        yhat = (p >= t).astype(int)
        tn, fp, fn, tp = confusion_matrix(y, yhat).ravel()
        sens = tp/(tp+fn) if (tp+fn) else np.nan
        spec = tn/(tn+fp) if (tn+fp) else np.nan
        prec = tp/(tp+fp) if (tp+fp) else np.nan
        acc  = (tp+tn)/(tp+tn+fp+fn)
        f1   = (2*prec*sens/(prec+sens)) if (prec>0 and sens>0) else np.nan
        J    = sens + spec - 1
        rows.append((t, acc, sens, spec, prec, f1, J, tp, fp, tn, fn))
    return pd.DataFrame(rows, columns=[
        'threshold','accuracy','sensitivity','specificity','precision','f1','youdenJ','tp','fp','tn','fn'
    ])





In [ ]:
grid = sweep_thresholds(bup_out, 'prob_remission_from_ARI_model')
# Example picks:
t_J   = grid.loc[grid['youdenJ'].idxmax(), 'threshold']      # maximizes sensitivity+specificity
t_F1  = grid.loc[grid['f1'].idxmax(), 'threshold']           # best F1
t_s80 = grid.iloc[(grid['sensitivity']-0.80).abs().idxmin()]['threshold']  # ~80% sens

In [37]:
grid = sweep_thresholds(ari_out, 'prob_remission_from_BUP_model')
# Example picks:
t_J   = grid.loc[grid['youdenJ'].idxmax(), 'threshold']      # maximizes sensitivity+specificity
t_F1  = grid.loc[grid['f1'].idxmax(), 'threshold']           # best F1
t_s80 = grid.iloc[(grid['sensitivity']-0.80).abs().idxmin()]['threshold']  # ~80% sens

In [9]:
#left join mean_prob from ON_bup_cog_predictions.csv by record_id onto preds_BUP_scored_by_ARI.csv and name the mean_prob column to mean_prob(onbup)
bup_out = bup_out.merge(
    pd.read_csv('ON_bup_clin_predictions.csv')[['record_id', 'mean_prob']],
    on='record_id', how='left'
)
bup_out = bup_out.rename(columns={'mean_prob': 'mean_prob(onbup)'})


In [10]:

# same for ARI
ari_out = ari_out.merge(
    pd.read_csv('ON_arp_clin_predictions.csv')[['record_id', 'mean_prob']],
    on='record_id', how='left'
)

In [11]:
ari_out = ari_out.rename(columns={'mean_prob': 'mean_prob(onari)'})


In [12]:
# concat ari_out and bup_out
all_out = pd.concat([bup_out, ari_out], ignore_index=True)

In [13]:
all_out.to_csv('preds_crosscohort_all_clin.csv', index=False)

In [10]:
##
baard_cohort=pd.read_csv('C:\\Users\\Hassan\\Documents\\Projects\\baard\\BAARD_cohort_predictions_clinonly.csv')

In [15]:
# left join the column MEDICATION_GROUP from master_df onto baard_cohort by record_id
baard_cohort = baard_cohort.merge(
    master_df[['record_id', 'medication_group']],
    on='record_id', how='left'
)   

## adding threshold columns to baard coohort predicitons csv

In [35]:
baard_cohort=pd.read_csv('C:\\Users\\Hassan\\Documents\\Projects\\baard\\BAARD_cohort_predictions.csv')

baard_cohort = baard_cohort.loc[:, :'p(remission_on_bup)']

In [11]:
#make columns for the thresholds using p(remission_on_arp) and p(remission_on_bup) make a function that thresholdes from 0.1 to 0.9 in steps of 0.1
def add_threshold_columns(df, prob_col, threshold_col_prefix='threshold'):
    thresholds = np.arange(0.1, 1.0, 0.01)  # 0.10, 0.11, ..., 0.99
    for t in thresholds:
        df[f'{threshold_col_prefix}_{t:.2f}'] = (df[prob_col] >= t).astype(int)
    return df


baard_cohort = add_threshold_columns(baard_cohort, 'p(remission_on_arp)', 'threshold_arp')
baard_cohort = add_threshold_columns(baard_cohort, 'p(remission_on_bup)', 'threshold_bup')


C:\Users\Hassan\AppData\Local\Temp\ipykernel_35992\303595855.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{threshold_col_prefix}_{t:.2f}'] = (df[prob_col] >= t).astype(int)
C:\Users\Hassan\AppData\Local\Temp\ipykernel_35992\303595855.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{threshold_col_prefix}_{t:.2f}'] = (df[prob_col] >= t).astype(int)
C:\Users\Hassan\AppData\Local\Temp\ipykernel_35992\303595855.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame

In [37]:
# print row 127
print(baard_cohort.iloc[127])

record_id             NaN
medication_group      NaN
age                   NaN
sex                   NaN
remission_status      NaN
                     ... 
threshold_bup_0.95      0
threshold_bup_0.96      0
threshold_bup_0.97      0
threshold_bup_0.98      0
threshold_bup_0.99      0
Name: 127, Length: 187, dtype: object


In [38]:
row_idx = 127
threshold_cols = [col for col in baard_cohort.columns if col.startswith("threshold_bup_") or col.startswith("threshold_arp_")]

for col in threshold_cols:
    if baard_cohort.loc[row_idx, col] == 0:
        col_without_row = baard_cohort[col].drop(index=row_idx)
        proportion = col_without_row.sum() / len(col_without_row)
        baard_cohort.loc[row_idx, col] = proportion


C:\Users\Hassan\AppData\Local\Temp\ipykernel_62640\102585320.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.9133858267716536' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  baard_cohort.loc[row_idx, col] = proportion
C:\Users\Hassan\AppData\Local\Temp\ipykernel_62640\102585320.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.905511811023622' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  baard_cohort.loc[row_idx, col] = proportion
C:\Users\Hassan\AppData\Local\Temp\ipykernel_62640\102585320.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.8976377952755905' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.


In [39]:
# Identify threshold columns
arp_cols = sorted([c for c in baard_cohort.columns if c.startswith("threshold_arp_")])
bup_cols = sorted([c for c in baard_cohort.columns if c.startswith("threshold_bup_")])

# Interleave them
interleaved_threshold_cols = []
for a, b in zip(arp_cols, bup_cols):
    interleaved_threshold_cols.extend([a, b])

# Keep the non-threshold columns in original order
non_threshold_cols = [c for c in baard_cohort.columns if c not in arp_cols + bup_cols]

# Reorder DataFrame
baard_cohort = baard_cohort[non_threshold_cols + interleaved_threshold_cols]


In [13]:
baard_cohort.to_csv('BAARD_cohort_predictions_threshold_clin.csv', index=False)

In [41]:
import numpy as np, pandas as pd
from sklearn.metrics import confusion_matrix, roc_auc_score, brier_score_loss

def sweep_metrics(df, prob_col, y_col, thresholds, model_label):
    # keep only rows with valid prob + label
    s = df[[prob_col, y_col]].copy()
    s[prob_col] = pd.to_numeric(s[prob_col], errors='coerce')
    s[y_col] = pd.to_numeric(s[y_col], errors='coerce')
    s = s.dropna()
    s = s[s[y_col].isin([0, 1])]

    if len(s) == 0:
        return pd.DataFrame()

    y = s[y_col].astype(int).values
    p = s[prob_col].astype(float).values

    # prob-based metrics (constant across thresholds)
    auc = roc_auc_score(y, p) if len(np.unique(y)) == 2 else np.nan
    brier = brier_score_loss(y, p)

    rows = []
    for t in sorted(thresholds):
        yhat = (p >= t).astype(int)
        tn, fp, fn, tp = confusion_matrix(y, yhat).ravel()
        sens = tp/(tp+fn) if (tp+fn) else np.nan
        spec = tn/(tn+fp) if (tn+fp) else np.nan
        prec = tp/(tp+fp) if (tp+fp) else np.nan
        npv  = tn/(tn+fn) if (tn+fn) else np.nan
        acc  = (tp+tn)/(tp+tn+fp+fn)
        f1   = (2*prec*sens/(prec+sens)) if (prec>0 and sens>0) else np.nan
        bal  = 0.5*(sens+spec) if (not np.isnan(sens) and not np.isnan(spec)) else np.nan

        rows.append({
            "model": model_label,
            "threshold": float(t),
            "n": len(y),
            "positives": int(y.sum()),
            "predicted_positives": int((yhat==1).sum()),
            "tp": int(tp), "fp": int(fp), "tn": int(tn), "fn": int(fn),
            "accuracy": acc, "sensitivity": sens, "specificity": spec,
            "precision": prec, "npv": npv, "f1": f1, "balanced_accuracy": bal,
            "auc": auc, "brier": brier
        })
    return pd.DataFrame(rows)

# pull the exact thresholds from your existing columns
arp_ts = {float(c.split('_')[-1]) for c in baard_cohort.columns if c.startswith("threshold_arp_")}
bup_ts = {float(c.split('_')[-1]) for c in baard_cohort.columns if c.startswith("threshold_bup_")}
# sensible fallbacks if not found
if not arp_ts: arp_ts = set(np.arange(0.10, 1.00, 0.01))
if not bup_ts: bup_ts = set(np.arange(0.10, 1.00, 0.01))

# compute
m_arp = sweep_metrics(baard_cohort, "p(remission_on_arp)", "remission_status", arp_ts, "ARP_model")
m_bup = sweep_metrics(baard_cohort, "p(remission_on_bup)", "remission_status", bup_ts, "BUP_model")

metrics = pd.concat([m_arp, m_bup], ignore_index=True)

# save
#out_path = "baard_threshold_metrics_clin.csv"
#metrics.to_csv(out_path, index=False)
#print(f"Saved: {out_path}")


In [12]:
baard_cohort = baard_cohort[~baard_cohort['record_id'].astype(str).str.startswith('CU')].dropna(subset=['remission_status'])

In [43]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score, brier_score_loss

def group_rates_at(df, t_arp, t_bup, y_col='remission_status',
                   arp_p='p(remission_on_arp)', bup_p='p(remission_on_bup)'):
    a1 = df[arp_p].astype(float) >= t_arp
    b1 = df[bup_p].astype(float) >= t_bup
    g1 = a1 & b1
    g2 = (~a1) & b1
    g3 = a1 & (~b1)
    g4 = (~a1) & (~b1)

    y = df[y_col].astype(int)

    def n_and_rate(mask):
        n = int(mask.sum())
        r = float(y[mask].mean()) if n > 0 else np.nan
        return n, r

    n1, r1 = n_and_rate(g1)
    n2, r2 = n_and_rate(g2)
    n3, r3 = n_and_rate(g3)
    n4, r4 = n_and_rate(g4)

    # relative size difference (info only)
    g23_rel_diff = (abs(n2 - n3) / max(n2, n3)) if max(n2, n3) > 0 else np.nan

    return dict(t_arp=float(t_arp), t_bup=float(t_bup),
                n1=n1, n2=n2, n3=n3, n4=n4,
                r1=r1, r2=r2, r3=r3, r4=r4,
                g23_rel_diff=g23_rel_diff)

def find_threshold_pairs_no_eq(baard_cohort,
                               g1_min=0.7, g23_min=0.15, g4_max=0.1,
                               min_n_each=5,
                               y_col='remission_status',
                               arp_p='p(remission_on_arp)', bup_p='p(remission_on_bup)'):
    arp_ts = sorted({float(c.replace('threshold_arp_', '')) 
                 for c in baard_cohort.columns if c.startswith('threshold_arp_')})
    bup_ts = sorted({float(c.replace('threshold_bup_', '')) 
                 for c in baard_cohort.columns if c.startswith('threshold_bup_')})


    rows = []
    for ta in arp_ts:
        for tb in bup_ts:
            m = group_rates_at(baard_cohort, ta, tb, y_col=y_col, arp_p=arp_p, bup_p=bup_p)

            # size floor
            if any(m[k] < min_n_each for k in ('n1','n2','n3','n4')):
                continue
            # rate constraints
            if not (m['r1'] >= g1_min): continue
            if not (m['r2'] >= g23_min and m['r3'] >= g23_min): continue
            if not (m['r4'] <= g4_max): continue

            rows.append(m)

    out = pd.DataFrame(rows)
    if out.empty:
        return out

    # Rank: prioritize higher min(r2,r3), then higher r1, then lower r4; show g23_rel_diff for context
    out['min_r23'] = out[['r2','r3']].min(axis=1)
    out = out.sort_values(['min_r23','r1','r4'], ascending=[False, False, True]).reset_index(drop=True)
    return out

# Run it
results = find_threshold_pairs_no_eq(
    baard_cohort,
    g1_min=0.75,
    g23_min=0.65,   # tweak if you want stricter/looser for groups 2&3
    g4_max=0.15,
    min_n_each=5    # raise if you want to avoid tiny groups
)

print(f"Found {len(results)} threshold pairs.")
results.head(10).round(3)

# Save all candidates
#results.to_csv("baard_threshold_pair_candidates_no_eq.csv", index=False)


Found 0 threshold pairs.


""


In [44]:
print(arp_ts)

{0.25, 0.27, 0.24, 0.26, 0.28, 0.5, 0.75, 0.51, 0.76, 0.79, 0.84, 0.21, 0.23, 0.22, 0.54, 0.38, 0.63, 0.88, 0.47, 0.72, 0.97, 0.18, 0.19, 0.42, 0.43, 0.44, 0.2, 0.45, 0.46, 0.41, 0.92, 0.69, 0.94, 0.15, 0.16, 0.17, 0.55, 0.8, 0.39, 0.64, 0.89, 0.48, 0.73, 0.98, 0.57, 0.82, 0.12, 0.13, 0.14, 0.59, 0.67, 0.83, 0.35, 0.6, 0.68, 0.85, 0.93, 0.11, 0.52, 0.77, 0.91, 0.36, 0.61, 0.86, 0.7, 0.95, 0.1, 0.56, 0.81, 0.4, 0.65, 0.9, 0.49, 0.74, 0.99, 0.58, 0.34, 0.29, 0.3, 0.31, 0.32, 0.33, 0.53, 0.78, 0.37, 0.62, 0.87, 0.71, 0.96, 0.66}


In [45]:
import numpy as np
import pandas as pd

def check_pair(df, t_arp=0.41, t_bup=0.41,
               y_col='remission_status',
               arp_p='p(remission_on_arp)', bup_p='p(remission_on_bup)'):
    s = df[[arp_p, bup_p, y_col]].copy()
    s[arp_p] = pd.to_numeric(s[arp_p], errors='coerce')
    s[bup_p] = pd.to_numeric(s[bup_p], errors='coerce')
    s[y_col] = pd.to_numeric(s[y_col], errors='coerce').astype('Int64')

    s = s.dropna(subset=[arp_p, bup_p, y_col])
    s[y_col] = s[y_col].astype(int)

    a1 = s[arp_p] >= t_arp
    b1 = s[bup_p] >= t_bup

    g1 = s.loc[a1 & b1, y_col]
    g2 = s.loc[(~a1) & b1, y_col]
    g3 = s.loc[a1 & (~b1), y_col]
    g4 = s.loc[(~a1) & (~b1), y_col]

    def rr(x): 
        return (x.mean() if len(x) else np.nan, len(x))
    r1,n1 = rr(g1); r2,n2 = rr(g2); r3,n3 = rr(g3); r4,n4 = rr(g4)

    print(f"(t_arp={t_arp:.2f}, t_bup={t_bup:.2f})  "
          f"G1 r={r1:.3f} n={n1} | G2 r={r2:.3f} n={n2} | "
          f"G3 r={r3:.3f} n={n3} | G4 r={r4:.3f} n={n4}")

check_pair(baard_cohort, 0.41, 0.41)


(t_arp=0.41, t_bup=0.41)  G1 r=0.789 n=19 | G2 r=0.500 n=8 | G3 r=0.385 n=39 | G4 r=0.167 n=54


In [46]:
import numpy as np, pandas as pd

def group_rates_at(df, t_arp, t_bup, y_col='remission_status',
                   arp_p='p(remission_on_arp)', bup_p='p(remission_on_bup)'):
    s = df[[arp_p, bup_p, y_col]].apply(pd.to_numeric, errors='coerce').dropna()
    y = s[y_col].astype(int)
    a1 = s[arp_p] >= t_arp
    b1 = s[bup_p] >= t_bup
    masks = {
        'g1': a1 & b1,
        'g2': (~a1) & b1,
        'g3': a1 & (~b1),
        'g4': (~a1) & (~b1),
    }
    out = {'t_arp': float(t_arp), 't_bup': float(t_bup)}
    for k, m in masks.items():
        n = int(m.sum())
        r = float(y[m].mean()) if n > 0 else np.nan
        out[f'n_{k}'] = n
        out[f'r_{k}'] = r
    out['min_r23'] = np.nanmin([out['r_g2'], out['r_g3']])
    return out

def find_pairs(df, g1_min=0.75, g23_min=0.65, g4_max=0.15, min_n_each=5,
               arp_p='p(remission_on_arp)', bup_p='p(remission_on_bup)'):
    # dense numeric grid (ignore column names entirely)
    thr = np.round(np.arange(0.10, 1.00, 0.01), 2)
    rows = []
    for ta in thr:
        for tb in thr:
            m = group_rates_at(df, ta, tb, arp_p=arp_p, bup_p=bup_p)
            if any(m[k] < min_n_each for k in ('n_g1','n_g2','n_g3','n_g4')): 
                continue
            if not (m['r_g1'] >= g1_min): continue
            if not (m['r_g2'] >= g23_min and m['r_g3'] >= g23_min): continue
            if not (m['r_g4'] <= g4_max): continue
            rows.append(m)
    return pd.DataFrame(rows)

def best_tradeoff(df, g1_min=0.75, g4_max=0.15, min_n_each=5,
                  arp_p='p(remission_on_arp)', bup_p='p(remission_on_bup)'):
    thr = np.round(np.arange(0.10, 1.00, 0.01), 2)
    rows = []
    for ta in thr:
        for tb in thr:
            m = group_rates_at(df, ta, tb, arp_p=arp_p, bup_p=bup_p)
            if any(m[k] < min_n_each for k in ('n_g1','n_g2','n_g3','n_g4')): 
                continue
            if not (m['r_g1'] >= g1_min): continue
            if not (m['r_g4'] <= g4_max): continue
            rows.append(m)
    out = pd.DataFrame(rows)
    if out.empty: return out
    # rank: maximize min(G2,G3), then maximize r_g1, then minimize r_g4
    return out.sort_values(['min_r23','r_g1','r_g4'],
                           ascending=[False, False, True]).reset_index(drop=True)

# 1) Try to meet your exact targets
exact = find_pairs(baard_cohort, g1_min=0.75, g23_min=0.65, g4_max=0.15, min_n_each=5)
print(f"Exact solutions found: {len(exact)}")
if not exact.empty:
    print(exact.head(10).round(3))

# 2) If none, get best trade-off (keeps G1≥0.75 and G4≤0.15, maximizes min(G2,G3))
trade = best_tradeoff(baard_cohort, g1_min=0.75, g4_max=0.15, min_n_each=5)
print("Top trade-offs (first rows):")
print(trade.head(10).round(3))

# Optional: save
# exact.to_csv("baard_exact_pairs.csv", index=False)
# trade.to_csv("baard_best_tradeoffs.csv", index=False)


C:\Users\Hassan\AppData\Local\Temp\ipykernel_62640\3715535658.py:21: RuntimeWarning: All-NaN axis encountered
  out['min_r23'] = np.nanmin([out['r_g2'], out['r_g3']])
C:\Users\Hassan\AppData\Local\Temp\ipykernel_62640\3715535658.py:21: RuntimeWarning: All-NaN axis encountered
  out['min_r23'] = np.nanmin([out['r_g2'], out['r_g3']])
C:\Users\Hassan\AppData\Local\Temp\ipykernel_62640\3715535658.py:21: RuntimeWarning: All-NaN axis encountered
  out['min_r23'] = np.nanmin([out['r_g2'], out['r_g3']])
C:\Users\Hassan\AppData\Local\Temp\ipykernel_62640\3715535658.py:21: RuntimeWarning: All-NaN axis encountered
  out['min_r23'] = np.nanmin([out['r_g2'], out['r_g3']])
C:\Users\Hassan\AppData\Local\Temp\ipykernel_62640\3715535658.py:21: RuntimeWarning: All-NaN axis encountered
  out['min_r23'] = np.nanmin([out['r_g2'], out['r_g3']])
C:\Users\Hassan\AppData\Local\Temp\ipykernel_62640\3715535658.py:21: RuntimeWarning: All-NaN axis encountered
  out['min_r23'] = np.nanmin([out['r_g2'], out['r_g3']]

Exact solutions found: 0


C:\Users\Hassan\AppData\Local\Temp\ipykernel_62640\3715535658.py:21: RuntimeWarning: All-NaN axis encountered
  out['min_r23'] = np.nanmin([out['r_g2'], out['r_g3']])
C:\Users\Hassan\AppData\Local\Temp\ipykernel_62640\3715535658.py:21: RuntimeWarning: All-NaN axis encountered
  out['min_r23'] = np.nanmin([out['r_g2'], out['r_g3']])
C:\Users\Hassan\AppData\Local\Temp\ipykernel_62640\3715535658.py:21: RuntimeWarning: All-NaN axis encountered
  out['min_r23'] = np.nanmin([out['r_g2'], out['r_g3']])
C:\Users\Hassan\AppData\Local\Temp\ipykernel_62640\3715535658.py:21: RuntimeWarning: All-NaN axis encountered
  out['min_r23'] = np.nanmin([out['r_g2'], out['r_g3']])
C:\Users\Hassan\AppData\Local\Temp\ipykernel_62640\3715535658.py:21: RuntimeWarning: All-NaN axis encountered
  out['min_r23'] = np.nanmin([out['r_g2'], out['r_g3']])
C:\Users\Hassan\AppData\Local\Temp\ipykernel_62640\3715535658.py:21: RuntimeWarning: All-NaN axis encountered
  out['min_r23'] = np.nanmin([out['r_g2'], out['r_g3']]

Top trade-offs (first rows):
   t_arp  t_bup  n_g1   r_g1  n_g2  r_g2  n_g3   r_g3  n_g4   r_g4  min_r23
0   0.34   0.41    21  0.762     6   0.5    45  0.378    48  0.146    0.378
1   0.28   0.41    22  0.773     5   0.4    51  0.373    42  0.119    0.373
2   0.29   0.41    22  0.773     5   0.4    51  0.373    42  0.119    0.373
3   0.30   0.41    22  0.773     5   0.4    51  0.373    42  0.119    0.373
4   0.33   0.41    22  0.773     5   0.4    46  0.370    47  0.149    0.370
5   0.31   0.41    22  0.773     5   0.4    49  0.367    44  0.136    0.367
6   0.32   0.41    22  0.773     5   0.4    49  0.367    44  0.136    0.367
7   0.27   0.41    22  0.773     5   0.4    53  0.358    40  0.125    0.358
8   0.34   0.40    24  0.750     6   0.5    42  0.357    48  0.146    0.357
9   0.28   0.40    25  0.760     5   0.4    48  0.354    42  0.119    0.354


C:\Users\Hassan\AppData\Local\Temp\ipykernel_62640\3715535658.py:21: RuntimeWarning: All-NaN axis encountered
  out['min_r23'] = np.nanmin([out['r_g2'], out['r_g3']])
C:\Users\Hassan\AppData\Local\Temp\ipykernel_62640\3715535658.py:21: RuntimeWarning: All-NaN axis encountered
  out['min_r23'] = np.nanmin([out['r_g2'], out['r_g3']])
C:\Users\Hassan\AppData\Local\Temp\ipykernel_62640\3715535658.py:21: RuntimeWarning: All-NaN axis encountered
  out['min_r23'] = np.nanmin([out['r_g2'], out['r_g3']])
C:\Users\Hassan\AppData\Local\Temp\ipykernel_62640\3715535658.py:21: RuntimeWarning: All-NaN axis encountered
  out['min_r23'] = np.nanmin([out['r_g2'], out['r_g3']])
C:\Users\Hassan\AppData\Local\Temp\ipykernel_62640\3715535658.py:21: RuntimeWarning: All-NaN axis encountered
  out['min_r23'] = np.nanmin([out['r_g2'], out['r_g3']])
C:\Users\Hassan\AppData\Local\Temp\ipykernel_62640\3715535658.py:21: RuntimeWarning: All-NaN axis encountered
  out['min_r23'] = np.nanmin([out['r_g2'], out['r_g3']]

In [47]:
import numpy as np
import pandas as pd

ARP_P = 'p(remission_on_arp)'
BUP_P = 'p(remission_on_bup)'
YCOL  = 'remission_status'

def group_stats(df, t_arp, t_bup, y_col=YCOL, arp_p=ARP_P, bup_p=BUP_P):
    s = df[[arp_p, bup_p, y_col]].apply(pd.to_numeric, errors='coerce').dropna()
    y = s[y_col].astype(int)
    a1 = s[arp_p] >= t_arp
    b1 = s[bup_p] >= t_bup
    masks = {
        'g1': a1 & b1,
        'g2': (~a1) & b1,
        'g3': a1 & (~b1),
        'g4': (~a1) & (~b1),
    }
    out = {'t_arp': float(t_arp), 't_bup': float(t_bup)}
    for k, m in masks.items():
        n = int(m.sum())
        r = float(y[m].mean()) if n > 0 else np.nan
        out[f'n_{k}'] = n
        out[f'r_{k}'] = r
    return out

def scan_grid(df, grid=None):
    if grid is None:
        grid = np.round(np.arange(0.10, 1.00, 0.01), 2)  # 0.10..0.99
    rows = []
    for ta in grid:
        for tb in grid:
            rows.append(group_stats(df, ta, tb))
    return pd.DataFrame(rows)

def apply_constraints(res,
                      g1_min=None, g23_min=None, g4_max=None,
                      min_n_each=None):
    mask = pd.Series(True, index=res.index)
    if min_n_each is not None:
        mask &= (res[['n_g1','n_g2','n_g3','n_g4']] >= min_n_each).all(axis=1)
    if g1_min is not None:
        mask &= res['r_g1'] >= g1_min
    if g23_min is not None:
        mask &= (res['r_g2'] >= g23_min) & (res['r_g3'] >= g23_min)
    if g4_max is not None:
        mask &= res['r_g4'] <= g4_max
    return res[mask].copy()

def best_group2_by_size(res):
    return res.sort_values(['n_g2','r_g2','r_g1','r_g4'],
                           ascending=[False, False, False, True]).head(10)

def best_group2_by_rate(res, min_n2=5):
    r = res[res['n_g2'] >= min_n2]
    return r.sort_values(['r_g2','n_g2','r_g1','r_g4'],
                         ascending=[False, False, False, True]).head(10)

# ---- run it ----
grid_results = scan_grid(baard_cohort)

# (A) Unconstrained: biggest Group 2 and highest Group 2 remission
print("Top Group 2 by SIZE (no constraints):")
print(best_group2_by_size(grid_results).round(3))

print("\nTop Group 2 by REMISSION RATE (no constraints, require n_g2>=5):")
print(best_group2_by_rate(grid_results, min_n2=5).round(3))

# (B) With your earlier goals as constraints (tweak as needed):
constrained = apply_constraints(grid_results,
                                g1_min=0.75,   # keep G1 high
                                g23_min=None,  # drop G2/G3 min if you want pure G2 focus
                                g4_max=None,   # keep G4 low
                                min_n_each=5)  # avoid tiny groups

print("\nTop Group 2 by SIZE (with constraints):")
print(best_group2_by_size(constrained).round(3))

print("\nTop Group 2 by REMISSION RATE (with constraints, n_g2>=5):")
print(best_group2_by_rate(constrained, min_n2=5).round(3))


def best_group1_allgroups(res):
    # keep only combos where every group has at least one subject
    r = res[(res[['n_g1','n_g2','n_g3','n_g4']] > 0).all(axis=1)].copy()
    # rank by best G1 remission; tie-break by larger G1, then lower G4 remission
    r = r.sort_values(['r_g1', 'n_g1', 'r_g4'], ascending=[False, False, True])
    return r

# ---- run it ----
grid_results = scan_grid(baard_cohort)

# All possible outcomes (no mins, no constraints), but requiring readings in G2, G3, G4
g1_ranked = best_group1_allgroups(grid_results)

print("All threshold pairs with non-empty G1..G4, ranked by Group 1 remission:")
print(g1_ranked.round(3))   # prints ALL rows, ranked

Top Group 2 by SIZE (no constraints):
      t_arp  t_bup  n_g1  r_g1  n_g2   r_g2  n_g3  r_g3  n_g4   r_g4
7380   0.92    0.1     0   NaN   104  0.394     0   NaN    16  0.125
7470   0.93    0.1     0   NaN   104  0.394     0   NaN    16  0.125
7560   0.94    0.1     0   NaN   104  0.394     0   NaN    16  0.125
7650   0.95    0.1     0   NaN   104  0.394     0   NaN    16  0.125
7740   0.96    0.1     0   NaN   104  0.394     0   NaN    16  0.125
7830   0.97    0.1     0   NaN   104  0.394     0   NaN    16  0.125
7920   0.98    0.1     0   NaN   104  0.394     0   NaN    16  0.125
8010   0.99    0.1     0   NaN   104  0.394     0   NaN    16  0.125
7110   0.89    0.1     1   1.0   103  0.388     0   NaN    16  0.125
7200   0.90    0.1     1   1.0   103  0.388     0   NaN    16  0.125

Top Group 2 by REMISSION RATE (no constraints, require n_g2>=5):
      t_arp  t_bup  n_g1  r_g1  n_g2  r_g2  n_g3  r_g3  n_g4   r_g4
7425   0.92   0.55     0   NaN     7   1.0     0   NaN   113  0.319
7

KeyboardInterrupt: 

In [ ]:
grid_results.to_csv("BAARD_threshold_grid_modality.csv", index=False)

##Plot auc curves all in one